# Langflow 实现代码检查

提交工作代码前需要找同事 code review。但是现在我们有大模型，不用大模型先 review 一番，总感觉不尊重同事。即使有隐私需求，本地部署大模型也不是难事。8G 显存就可以轻松运行 `deepseek-r1:14b`，这就够用了。

实现一个这样的工作流特别简单，最终完成的工作流如下：

![](./img/code_review.jpg)

## 1. 提示词模板

要改一下提示词模板。打开 Prompt 组件，修改 Template 如下：

```
下面是我的代码：

{content}

---

请检查这份代码是否有错误，如果没有，请提一些改进意见。

检查结果：
```

## 2. 模型部分

可以用 Ollama 模型。如果代码不是很敏感，也可以像我一样，用 deepseek api。很便宜的。在下面的链接注册，获取 API_KEY.

DeepSeek Platform: [platform.deepseek.com](https://platform.deepseek.com/)

## 3. 本地提交代码

需要一段本地代码，来指定你要检查的文件。并且读取文件内容，调用 langflow api，把文件内容传给 langflow。这部分代码我写在 [./code_reviewer/review.py](./code_reviewer/review.py) 中了。

```python
# install:
#   pip install click requests python-dotenv -i https://mirrors.cloud.tencent.com/pypi/simple/
# 
# Linux setting:
#   vim ~/.bashrc
#   alias review="python /path/to/your/review.py"
# 
# usage:
#   review -f ./your_code.py
#
# simple usage:
#   python3 review.py -f ./your_code.py


import os
import requests
import click

from dotenv import load_dotenv


# 加载敏感参数
load_dotenv()
langflow_app_id = os.getenv("LANGFLOW_APP_ID").strip()


# 参数
URL = f"http://127.0.0.1:7860/api/v1/run/{langflow_app_id}?stream=false"
HEADERS = {'Content-Type': 'application/json'}


def get_content(file_path):
    if not os.path.exists(file_path):
        print(f"Error: can not find {file_path}")
        return None

    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    return content


def review(message, url, headers):
    data = {
        "input_value": message,
        "output_type": "chat",
        "input_type": "chat",
        "tweaks": {
            "ChatInput-aYCPq": {},
            "Prompt-RbwEW": {},
            "ChatOutput-KIRYB": {},
            "CustomComponent-vtvSX": {}
        }
    }

    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        print(f'status_code: {response.status_code}')

    return None


def gen_review(filepath):
    filepath = filepath.strip()
    if not filepath:
        print('You should use -f to specify the file path')

    content = get_content(filepath)
    if content is not None:
        response = review(message=content,
                          url=URL,
                          headers=HEADERS)
        res = response["outputs"][0]["outputs"][0]["results"]["message"]["text"]
        return res

    return None


@click.command()
@click.option('-f', '--filepath', required=True, type=click.STRING, help='Specify the file path')
def app(filepath):
    return gen_review(filepath)


if __name__ == '__main__':
    res = app(standalone_mode=False)
    print(f"检查报告出炉的说 (・ω< )★\n\n{res}")

```